In [22]:
from kubernetes import client, config
import subprocess 
import json 
import logging 
import os
from flask import Flask, make_response, Response
from datetime import datetime, timedelta
from flask_caching import Cache
from prometheus_flask_exporter import PrometheusMetrics

def collect_seeds():
    static_seeds = [
        "/dns4/seed.piconbello.com/tcp/10001/p2p/12D3KooWRFac2AztcTeen2DYNwnTrmVBvwNDsRiFpDVdTkwdFAHP",
        "/dns4/seed.minaexplorer.com/tcp/8302/p2p/12D3KooWR7coZtrMHvsgsfiWq2GESYypac3i29LFGp6EpbtjxBiJ",
        "/dns4/mina-seed-1.zkvalidator.com/tcp/8302/p2p/12D3KooWSR7LMBSfEk3LQUudmsX27yuRHe9NUxwLumurGF5P1MNS",
        "/dns4/seed-2.piconbello.com/tcp/10001/p2p/12D3KooWMPxTu24mCpi3TwmkU4fJk7a8TQ4agFZeTHQRi8KCc3nj"
    ]
    dynamic_seeds = []
    namespace = os.environ.get("SEED_NAMESPACE", "mainnet")
    config.load_kube_config()
    
    # Get pods with seed in name 
    v1 = client.CoreV1Api()
    logging.info("Listing pods with their IPs:")
    ret = v1.list_pod_for_all_namespaces(watch=False)
    seed_pods = [ pod for pod in ret.items if (pod.metadata.namespace == namespace and "seed" in pod.metadata.name) ]

    if seed_pods == []:
        logging.error("No dynamic seed pods found 😥")
        return static_seeds

    else:
        for pod in seed_pods:
            pod_name = pod.metadata.name

            # Get the libp2p ID and port info
            pod_status = get_pod_status(pod_name, namespace)
            network_info = pod_status["addrs_and_ports"]
            peer_info = network_info["peer"]
            multiaddr = f"/ip4/{network_info['external_ip']}/tcp/{peer_info['libp2p_port']}/p2p/{peer_info['peer_id']}"
            dynamic_seeds.append(multiaddr)
    return dynamic_seeds + static_seeds

In [17]:
def get_pod_status(pod_name, namespace):
    status_command = f"kubectl exec --namespace {namespace} {pod_name} -- mina client status --json"
    logging.debug(f"Running command: {status_command}")
    get_status = subprocess.run(status_command.split(), stdout=subprocess.PIPE, text=True,)
    # skip non-json first line of output
    try:
        stdout_raw = get_status.stdout.split("\n")[1]
    except IndexError:
        # We didnt get a response
        logging.error(f"Error, didnt't get a parsable response.")
        logging.debug(f"stdout: {get_status.stdout}, stderr: {get_status.stderr}")
        return {}
    try:
        status_response = json.loads(get_status.stdout)

        if "error" in status_response:
            logging.error(f"Error in 'mina client status' call: {status_response}")
            return status_response
        else:
            return status_response
    except json.JSONDecodeError:
        logging.error(f"Error, didnt't get a parsable response. Instead got: {stdout_raw}")
        logging.debug(f"stdout: {status_response.stdout}, stderr: {status_response.stderr}")
        return {}

In [23]:
#Web Server
flask_config = {
    "DEBUG": True,          # some Flask specific configs
    "CACHE_TYPE": "SimpleCache",  # Flask-Caching related configs
    "CACHE_DEFAULT_TIMEOUT": 300
}
app = Flask(__name__)
app.config.from_mapping(flask_config)

metrics = PrometheusMetrics(app)
metrics.info('app_info', 'Application info', version='0.0.1')

cache = Cache(app)
log_level = os.environ.get("LOG_LEVEL", "DEBUG")

@cache.cached(timeout=300)
@app.route("/", methods = ['GET'])
def dynamic_seed():
    seeds = collect_seeds()
    content = "\n".join(seeds)
    response = Response(content, status=200, mimetype='text/plain')
    return response

@app.before_first_request
def gcloud_auth():
    
    GOOGLE_APPLICATION_CREDENTIALS = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS", None)
    
    if GOOGLE_APPLICATION_CREDENTIALS:
        activate_command = f"gcloud auth activate-service-account --key-file {GOOGLE_APPLICATION_CREDENTIALS}"
        logging.debug(f"Running command: {activate_command}")
        activate = subprocess.run(activate_command.split(), stdout=subprocess.PIPE, text=True,)
        stdout_raw = activate.stdout
        logging.debug(stdout_raw)
   
    CLUSTER_NAME = os.environ.get("CLUSTER_NAME", "sushi-dev")
    CLUSTER_REGION = os.environ.get("CLUSTER_REGION", "us-central1-c")
    GCLOUD_PROJECT = os.environ.get("GCLOUD_PROJECT", "sushivalidator")
    
    # get cluster kubeconfig 
    kubectl_command = f"gcloud container clusters get-credentials {CLUSTER_NAME} --region {CLUSTER_REGION} --project {GCLOUD_PROJECT}"
    # kubeconfig entry generated for sushi-dev.
    logging.debug(f"Running command: {kubectl_command}")
    auth_login = subprocess.run(kubectl_command.split(), stdout=subprocess.PIPE, text=True,)
    stdout_raw = auth_login.stdout
    logging.debug(stdout_raw)
    if "kubeconfig entry generated for" in stdout_raw:
        return True
    else: 
        return False

if __name__ == "__main__":
    logging.basicConfig(level=log_level)
    app.run(debug=True, host="0.0.0.0", port=int(os.environ.get("PORT", 8080)))

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use